## Costruzione del dataset contenente per ogni campione la media della TIC per ogni minuto

### Costruzione del dataframe

In [1]:
import pandas as pd
import pyopenms as pms
import matplotlib.pyplot as plt

Determination of memory status is not supported on this 
 platform, measuring for memoryleaks will never fail


#### Creazione un primo df di appoggio contenente origine - cultivar e percorso del file mzdata VOC del campione

In [2]:
tabella_file = pd.read_excel('/Users/manuelplacella/Desktop/Progetto IA/voo-geographical-origin/campagna-olearia_21-22/annotazioni.xlsx')
path_base = '/Users/manuelplacella/Desktop/Progetto IA/voo-geographical-origin/campagna-olearia_21-22/VOCs-mzdata/'
elenco_origini = []
elenco_cultivar = []
elenco_percorsi = []
for indice_riga, riga in tabella_file.iterrows():
    if str(riga['VOCs']).strip() == 'Sì':
        elenco_origini.append(str(riga['Origine']).strip().lower())
        elenco_cultivar.append(str(riga['CV']).strip().lower())
        elenco_percorsi.append(path_base + str(riga['Sample']).strip() + '.mzdata.xml')
df_file = pd.DataFrame({'Origine': elenco_origini, 'Cultivar': elenco_cultivar, 'File': elenco_percorsi})
df_file

,Origine,Cultivar,File
0,spagna,arbequina,/Users/manuelplacella/Desktop/Progetto IA/voo-...
1,spagna,arbequina,/Users/manuelplacella/Desktop/Progetto IA/voo-...
2,portogallo,arbequina,/Users/manuelplacella/Desktop/Progetto IA/voo-...
3,portogallo,arbequina,/Users/manuelplacella/Desktop/Progetto IA/voo-...
4,portogallo,arbequina,/Users/manuelplacella/Desktop/Progetto IA/voo-...
...,...,...,...
212,grecia-creta,psilolia,/Users/manuelplacella/Desktop/Progetto IA/voo-...
213,grecia-creta,psilolia,/Users/manuelplacella/Desktop/Progetto IA/voo-...
214,grecia-creta,psilolia,/Users/manuelplacella/Desktop/Progetto IA/voo-...
215,grecia-creta,psilolia,/Users/manuelplacella/Desktop/Progetto IA/voo-...


#### Definizione di una funzione che per ogni experiment (corrispondente ad un file) mi calcola la corrente ionica totale (TIC) e calcola la media della TIC dal minuti min_range al minuto max_range

In [3]:
min_range = 1
max_range = 41
def calcolaMedie(exp):
    tic = exp.calculateTIC()
    retention_time, intensity = tic.get_peaks()
    lista_medie = []
    pos_t_i = 0
    for min in range(min_range,max_range):
        somma = 0
        cont = 0
        while retention_time[pos_t_i] < 60*min :
            somma += intensity[pos_t_i]
            pos_t_i += 1
            cont += 1
        mean_intensity = somma/cont
        lista_medie.append(mean_intensity)
    return lista_medie

### Costruzione del dataset di lavoro inserendo i target 'origine' e 'cultivar' e 40 feature corrispondenti alla corrente ionica totale media in ogni minuto per ogni campione

In [4]:
colonna_origini = []
colonna_cultivar = []
colonna_valori = []
for index, row in df_file.iterrows():
    print('Campione ' + str(index))
    colonna_origini.append(row['Origine'])
    colonna_cultivar.append(row['Cultivar'])
    exp = pms.MSExperiment()
    pms.MzDataFile().load(str(row.File), exp)
    colonna_valori.append(calcolaMedie(exp))
df = pd.DataFrame(colonna_valori, columns=['minuto {}'.format(i+1) for i in range(max_range-1)])
# Aggiungo la colonna cultivar
df.insert(0,'Cultivar', colonna_cultivar)
# Aggiungo la colonna origine
df.insert(0,'Origine', colonna_origini)

Campione 0
Campione 1
Campione 2
Campione 3
Campione 4
Campione 5
Campione 6
Campione 7
Campione 8
Campione 9
Campione 10
Campione 11
Campione 12
Campione 13
Campione 14
Campione 15
Campione 16
Campione 17
Campione 18
Campione 19
Campione 20
Campione 21
Campione 22
Campione 23
Campione 24
Campione 25
Campione 26
Campione 27
Campione 28
Campione 29
Campione 30
Campione 31
Campione 32
Campione 33
Campione 34
Campione 35
Campione 36
Campione 37
Campione 38
Campione 39
Campione 40
Campione 41
Campione 42
Campione 43
Campione 44
Campione 45
Campione 46
Campione 47
Campione 48
Campione 49
Campione 50
Campione 51
Campione 52
Campione 53
Campione 54
Campione 55
Campione 56
Campione 57
Campione 58
Campione 59
Campione 60
Campione 61
Campione 62
Campione 63
Campione 64
Campione 65
Campione 66
Campione 67
Campione 68
Campione 69
Campione 70
Campione 71
Campione 72
Campione 73
Campione 74
Campione 75
Campione 76
Campione 77
Campione 78
Campione 79
Campione 80
Campione 81
Campione 82
Campione 83
Ca

In [5]:
df

,Origine,Cultivar,minuto 1,minuto 2,minuto 3,minuto 4,minuto 5,minuto 6,minuto 7,minuto 8,...,minuto 31,minuto 32,minuto 33,minuto 34,minuto 35,minuto 36,minuto 37,minuto 38,minuto 39,minuto 40
0,spagna,arbequina,269838.435185,214440.194656,404471.480916,9.718674e+05,214243.927481,372427.763359,440968.820611,4.301475e+05,...,260476.667939,177054.813688,237462.484733,255932.961832,411766.759542,451533.547529,656340.763359,917184.419847,1.351759e+06,2.129662e+06
1,spagna,arbequina,240197.685185,195479.748092,322672.637405,2.361754e+06,334500.408397,512092.049618,503167.267176,8.102410e+05,...,118197.698473,134956.091255,145618.702290,206248.954198,251911.530534,345735.072243,475011.442748,680705.267176,1.034518e+06,1.630518e+06
2,portogallo,arbequina,228354.759259,180756.251908,297156.332061,7.448458e+05,306539.671756,607468.580153,857707.442748,9.320031e+05,...,131935.664122,141273.167300,162876.175573,232173.099237,363278.255725,378103.456274,529309.564885,737379.835878,1.107545e+06,1.744317e+06
3,portogallo,arbequina,243738.574074,196577.687023,343740.503817,7.543858e+05,257065.568702,504839.583969,735639.935115,1.166171e+06,...,140585.106870,142509.600760,151270.954198,204724.114504,303272.633588,353698.722433,493036.515267,691654.187023,1.039010e+06,1.640625e+06
4,portogallo,arbequina,238562.805556,192566.801527,351229.049618,8.487872e+05,309813.549618,484581.519084,712942.324427,8.926326e+05,...,119389.702290,137154.882129,149447.442748,202541.198473,302269.061069,350110.680608,508800.931298,687794.148855,1.044465e+06,1.637805e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,grecia-creta,psilolia,282523.444444,223229.706107,399778.286260,1.005979e+06,279335.179389,451478.332061,502599.847328,4.195175e+05,...,164768.759542,161470.391635,179481.148855,210488.851145,286966.870229,385167.908745,532677.561069,765502.019084,1.132137e+06,1.780847e+06
213,grecia-creta,psilolia,284255.787037,226056.687023,389888.290076,1.000605e+06,306786.870229,454201.702290,500089.835878,5.298876e+05,...,172875.576336,168867.197719,187645.106870,217010.572519,293693.927481,394577.577947,543966.587786,780407.083969,1.155864e+06,1.820886e+06
214,grecia-creta,psilolia,285180.407407,226563.778626,459416.194656,1.092579e+06,268107.988550,410765.385496,499917.622137,4.419611e+05,...,223415.633588,209212.011407,184175.694656,216538.877863,297940.992366,381341.353612,539286.900763,793366.889313,1.155609e+06,1.824194e+06
215,grecia-creta,psilolia,274340.259259,215180.129771,359988.148855,1.355812e+06,251213.251908,516001.374046,371830.988550,3.362284e+05,...,262879.282443,201107.030418,176834.736641,203410.248092,283698.583969,358531.680608,508422.790076,759418.549618,1.103785e+06,1.743720e+06


### Salvataggio del dataframe in un csv da utilizzare poi per testare i vari algoritmi di machine learning

In [6]:
df['Cultivar'].value_counts()

picual                  50
psilolia                44
koroneiki               32
arbequina               31
coratina                22
chemlali                 8
hojiblanca               7
manzanilla               5
morisca                  3
picudo                   3
moraiolo                 2
frantoio                 1
carolea                  1
blend                    1
leccio del corno         1
chemlali 90%             1
maurino                  1
neroliva                 1
ogliarola                1
carnenzuela              1
koroneiki & neroliva     1
Name: Cultivar, dtype: int64

In [7]:
df['Origine'].value_counts()

spagna                  91
grecia-creta            44
italia                  29
grecia - peloponneso    24
portogallo              10
grecia                  10
tunisia                  9
Name: Origine, dtype: int64

In [8]:
df.to_csv('../dataset/df_media_TIC_VOCs_min_1-40.csv', index=False)